In [1]:
# imports and setup 
from bs4 import BeautifulSoup
# you can use either of these libraries to get html from a website
import requests
import urllib.request
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re

import pandas as pd
import scipy as sc
import numpy as np
import time
import statsmodels.formula.api as sm

import matplotlib.pyplot as plt 
plt.style.use('ggplot')
%matplotlib inline 
plt.rcParams['figure.figsize'] = (10, 6) 
from string import ascii_lowercase

In [2]:
#Get all the pages for each letter
url = "https://www.basketball-reference.com/players"
all_letters = ""
html = {}
i = 0
for x in ascii_lowercase:
    if x == 'x':
        pass
    else:
        link = url + '/' + str(x) + '/'
        with urllib.request.urlopen(link) as response:
            html[i] = response.read()
            html[i] = html[i].decode('utf-8')
        all_letters = all_letters + str(html[i])
        i += 1

In [3]:
with open('all_letters.html','w') as new_file:
    new_file.write(all_letters)

In [4]:
letters_soup = BeautifulSoup(open('all_letters.html',encoding = 'utf-8'), "html.parser")

In [5]:
#Get every players' link
links = []
url = "https://www.basketball-reference.com"
for x in letters_soup.find_all(class_="left "):
    if x.get("data-stat") == "player":
        link_tail = x.find("a").get("href")
        link = url + str(link_tail)
        links.append(link)


In [ ]:
rookie = []
columns_pergame_r = ['Name','link','Year','Age','Team','League','Position','Games Played','Games Started','MinPerGame','FGPerGame','FGAPerGame','FG%PerGame','3pPerGame','3PAPerGame','3p%PerGame','2pPerGame','2pAPerGame','2p%PerGame','eFG%PerGame','FTPerGame','FTAPerGame','FT%PerGame','ORGPerGame','DRBPerGame','TotalRebPerGame','AstPerGame','StealPerGame','BlkPerGame','TOVPerGame','FoulsPerGame','PtsPerGame']
df_pergame = pd.DataFrame(columns = columns_pergame_r)
columns_per100_r = ['Name','link','Year','Age','Team','League','Position','Games Played','Games Started','TotalMin','FGPer100Poss','FGAPer100Poss','FG%Per100Poss','3pPer100Poss','3PAPer100Poss','3p%Per100Poss','2pPer100Poss','2pAPer100Poss','2p%Per100Poss','FTPer100Poss','FTAPer100Poss','FT%Per100Poss','ORGPer100Poss','DRBPer100Poss','TotalRebPer100Poss','AstPer100Poss','StealPer100Poss','BlkPer100Poss','TOVPer100Poss','FoulsPer100Poss','PtsPer100Poss','blank','ORtg','DRtg']
df_per100 = pd.DataFrame(columns = columns_per100_r)
columns_advanced_r = ['Name','link','Year','Age','Team','League','Position','Games Played','TotalMin','PER','TS%','3PAr','FTr','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%','blank','OWS','DWS','WS','WS/48','blank','OBPM','DBPM','BPM','VORP']
df_advanced = pd.DataFrame(columns = columns_advanced_r)

career = []
columns_pergame_c = ['Name','link','Year','blank','blank','League','blank','Games Played','Games Started','MinPerGame','FGPerGame','FGAPerGame','FG%PerGame','3pPerGame','3PAPerGame','3p%PerGame','2pPerGame','2pAPerGame','2p%PerGame','eFG%PerGame','FTPerGame','FTAPerGame','FT%PerGame','ORGPerGame','DRBPerGame','TotalRebPerGame','AstPerGame','StealPerGame','BlkPerGame','TOVPerGame','FoulsPerGame','PtsPerGame']
career_pergame = pd.DataFrame(columns = columns_pergame_c)
columns_per100_c = ['Name','link','Year','blank','blank','League','blank','Games Played','Games Started','TotalMin','FGPer100Poss','FGAPer100Poss','FG%Per100Poss','3pPer100Poss','3PAPer100Poss','3p%Per100Poss','2pPer100Poss','2pAPer100Poss','2p%Per100Poss','FTPer100Poss','FTAPer100Poss','FT%Per100Poss','ORGPer100Poss','DRBPer100Poss','TotalRebPer100Poss','AstPer100Poss','StealPer100Poss','BlkPer100Poss','TOVPer100Poss','FoulsPer100Poss','PtsPer100Poss','blank','ORtg','DRtg']
career_per100 = pd.DataFrame(columns = columns_per100_c)
columns_advanced_c = ['Name','link','Year','blank','blank','League','blank','Games Played','TotalMin','PER','TS%','3PAr','FTr','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%','blank','OWS','DWS','WS','WS/48','blank','OBPM','DBPM','BPM','VORP']
career_advanced = pd.DataFrame(columns = columns_advanced_c)

def get_num(x):
    return int(''.join(ele for ele in x if ele.isdigit()))

awards = []
columns_a = ['Name','link','MVPs','All-Star Games','All-NBA Teams','All-Rookie Team','ROY']
career_awards = pd.DataFrame(columns = columns_a)
zeros = [0,0,0,0,0,0,0]

columns_dp = ['Name','link','Draft_Pick']
draft_pick = pd.DataFrame(columns = columns_dp)
zeros1 = [0,0,0]


In [35]:
# Get everything

j=1
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(chrome_options=chrome_options)
start_time = time.time()
for i in range(3646, len(links)):
    j+=1
    iter_time = time.time()
    url = links[i]
    driver.get(url)
    players_soup = BeautifulSoup(driver.page_source,'html.parser')
    for x in players_soup.find_all(id="per_game"):
        for q in players_soup.find("h1"):
            rookie.append(q)
            career.append(q)
        rookie.append(url[47:-5])
        career.append(url[47:-5])
        for y in x.find_all("tr")[1]:
            rookie.append(y.get_text())
        if len(rookie) == 32:
            df_pergame.loc[i] = rookie  ##Get rookie_pergame
        rookie = []
        for r in x.find_all("tfoot"):
            for s in r.find_all("tr")[0]:
                career.append(s.get_text())
        if len(career) == 32:
            career_pergame.loc[i] = career #Get career_pergame
        career = []
        
    for x in players_soup.find_all(id="per_poss"):
        for q in players_soup.find("h1"):
            rookie.append(q)
            career.append(q)
        rookie.append(url[47:-5])
        career.append(url[47:-5])
        for y in x.find_all("tr")[1]:
            rookie.append(y.get_text())
        if len(rookie) == 34:
            df_per100.loc[i] = rookie #Get rookie_per100
        rookie = []  
        
        for z in x.find_all("tfoot"):
            for y in z.find_all("tr")[0]:
                career.append(y.get_text())
        if len(career) == 34:
            career_per100.loc[i] = career #Get career_per100
        career = []
        
    for x in players_soup.find_all(id="advanced"):
        for q in players_soup.find("h1"):
            rookie.append(q)
            career.append(q)
        rookie.append(url[47:-5])
        career.append(url[47:-5])
        for y in x.find_all("tr")[1]:
            rookie.append(y.get_text())
        if len(rookie) == 31:
            df_advanced.loc[i] = rookie #Get rookie_advanced
        rookie = []
        
        for z in x.find_all("tfoot"):
            for y in z.find_all("tr")[0]:
                career.append(y.get_text())
        if len(career) == 31:
            career_advanced.loc[i] = career #Get career_advanced
        career = []
        
    for x in players_soup.find_all(id="info"): #Get awards
        if x.find_all(id="bling") != []:
            career_awards.loc[i] = zeros
            career_awards['link'].loc[i] = url[47:-5]
            for y in x.find_all("li"):
                for q in players_soup.find("h1"):
                    career_awards['Name'].loc[i] = q
                career_awards['link'].loc[i] = url[47:-5]
                result = y.get_text()
                if "x MVP" in result:
                    career_awards['MVPs'].loc[i] = get_num(result)
                if re.findall(r'\d\sMVP',result) != []:
                    career_awards['MVPs'].loc[i] = 1
                if "All Star" in result:
                    career_awards['All-Star Games'].loc[i] = get_num(result)
                if "All-NBA" in result:
                    if get_num(result) > 30: #For when it says 2018 All-NBA, this means they only won once. We don't want it saying they won 2018 times
                        career_awards['All-NBA Teams'].loc[i] = 1
                    else:
                        career_awards['All-NBA Teams'].loc[i] = get_num(result)
                if "All-Rookie" in result:
                    career_awards['All-Rookie Team'].loc[i] = 1
                if "ROY" in result:
                    career_awards['ROY'].loc[i] = 1
                result = ""
        else:
            career_awards.loc[i] = zeros
            career_awards['link'].loc[i] = url[47:-5]
            for q in players_soup.find("h1"):
                career_awards['Name'].loc[i] = q
            
    for x in players_soup.find_all(id="meta"): #Get their draft position
        draft_pick.loc[i] = zeros1
        result = x.get_text()
        for q in players_soup.find("h1"):
            draft_pick['Name'].loc[i] = q
        draft_pick['link'].loc[i] = url[47:-5]
        if re.findall(r'\d{1,3}\w\w\soverall',result) != []:
            pick = re.findall(r'\d{1,3}\w\w\soverall',result)[0]
            draft_pick['Draft_Pick'].loc[i] = re.findall(r'\d{1,3}',pick)[0]
        else: 
            draft_pick['Draft_Pick'].loc[i] = np.nan
    print("Iteration: " + str(i) + ". Iteration Time: " + str(time.time()-iter_time) + ". Average time per iteration: " + str((time.time()-start_time)/(j)))
print("Total Time: " + str(time.time()-start_time) + ". Average time per iteration: " + str((time.time()-start_time)/j))
driver.close()

Iteration: 3646. Iteration Time: 12.91002607345581. Average time per iteration: 6.458523511886597
Iteration: 3647. Iteration Time: 2.9566471576690674. Average time per iteration: 5.291608015696208
Iteration: 3648. Iteration Time: 2.881181240081787. Average time per iteration: 4.689065217971802
Iteration: 3649. Iteration Time: 2.286022901535034. Average time per iteration: 4.208488178253174
Iteration: 3650. Iteration Time: 3.537619113922119. Average time per iteration: 4.096697648366292
Iteration: 3651. Iteration Time: 1.6757612228393555. Average time per iteration: 3.750874859946115
Iteration: 3652. Iteration Time: 1.7108616828918457. Average time per iteration: 3.495889127254486
Iteration: 3653. Iteration Time: 2.0431151390075684. Average time per iteration: 3.33448855082194
Iteration: 3654. Iteration Time: 1.3684139251708984. Average time per iteration: 3.137895202636719
Iteration: 3655. Iteration Time: 1.0504121780395508. Average time per iteration: 2.9481360045346348
Iteration: 365

Iteration: 3729. Iteration Time: 1.5816302299499512. Average time per iteration: 1.9798990333781523
Iteration: 3730. Iteration Time: 2.231806993484497. Average time per iteration: 1.9828294765117556
Iteration: 3731. Iteration Time: 2.285507917404175. Average time per iteration: 1.9863098983106942
Iteration: 3732. Iteration Time: 4.069800853729248. Average time per iteration: 2.0099876685576006
Iteration: 3733. Iteration Time: 1.5007908344268799. Average time per iteration: 2.0042685605167003
Iteration: 3734. Iteration Time: 1.5184299945831299. Average time per iteration: 1.9988717317581177
Iteration: 3735. Iteration Time: 1.1042637825012207. Average time per iteration: 1.9890423502240862
Iteration: 3736. Iteration Time: 2.4565670490264893. Average time per iteration: 1.994125783443451
Iteration: 3737. Iteration Time: 1.0609591007232666. Average time per iteration: 1.9840930738756735
Iteration: 3738. Iteration Time: 1.306422233581543. Average time per iteration: 1.9768850752647886
Itera

Iteration: 3812. Iteration Time: 2.4211173057556152. Average time per iteration: 1.9924973036561693
Iteration: 3813. Iteration Time: 2.331486940383911. Average time per iteration: 1.9945041117583506
Iteration: 3814. Iteration Time: 1.1718220710754395. Average time per iteration: 1.989665571381064
Iteration: 3815. Iteration Time: 2.635378122329712. Average time per iteration: 1.993442322078504
Iteration: 3816. Iteration Time: 1.361968755722046. Average time per iteration: 1.989771733450335
Iteration: 3817. Iteration Time: 1.4182751178741455. Average time per iteration: 1.9864690000611234
Iteration: 3818. Iteration Time: 1.8347549438476562. Average time per iteration: 1.9855978214877776
Iteration: 3819. Iteration Time: 1.7000408172607422. Average time per iteration: 1.9839667892456054
Iteration: 3820. Iteration Time: 1.600691795349121. Average time per iteration: 1.9817897839979692
Iteration: 3821. Iteration Time: 2.8582100868225098. Average time per iteration: 1.986742027735306
Iteratio

Iteration: 3895. Iteration Time: 1.669511318206787. Average time per iteration: 1.9203105495270505
Iteration: 3896. Iteration Time: 2.8976309299468994. Average time per iteration: 1.9241893490155537
Iteration: 3897. Iteration Time: 1.738847017288208. Average time per iteration: 1.9234573162591504
Iteration: 3898. Iteration Time: 3.644644021987915. Average time per iteration: 1.9302343186431044
Iteration: 3899. Iteration Time: 1.6667520999908447. Average time per iteration: 1.9292015795614206
Iteration: 3900. Iteration Time: 1.835629940032959. Average time per iteration: 1.9288365738466382
Iteration: 3901. Iteration Time: 1.3512849807739258. Average time per iteration: 1.9265898322316923
Iteration: 3902. Iteration Time: 1.0785980224609375. Average time per iteration: 1.9233037288798842
Iteration: 3903. Iteration Time: 2.5831210613250732. Average time per iteration: 1.9258517289253736
Iteration: 3904. Iteration Time: 2.5817737579345703. Average time per iteration: 1.9283750460698055
Iter

Iteration: 3978. Iteration Time: 1.5680408477783203. Average time per iteration: 1.9071308595691612
Iteration: 3979. Iteration Time: 1.6893091201782227. Average time per iteration: 1.9064810176393878
Iteration: 3980. Iteration Time: 1.600614309310913. Average time per iteration: 1.9055711371558053
Iteration: 3981. Iteration Time: 2.2390830516815186. Average time per iteration: 1.906561183646454
Iteration: 3982. Iteration Time: 2.096614122390747. Average time per iteration: 1.9071238400668082
Iteration: 3983. Iteration Time: 3.9612929821014404. Average time per iteration: 1.9131837045900237
Iteration: 3984. Iteration Time: 2.5053870677948. Average time per iteration: 1.9149259174571318
Iteration: 3985. Iteration Time: 1.9353320598602295. Average time per iteration: 1.9149861552498557
Iteration: 3986. Iteration Time: 1.2663960456848145. Average time per iteration: 1.913090111219395
Iteration: 3987. Iteration Time: 1.4484150409698486. Average time per iteration: 1.9117358363404566
Iterati

...

Iteration: 4227. Iteration Time: 1.9460668563842773. Average time per iteration: 1.9657835158843733
Iteration: 4228. Iteration Time: 1.0971941947937012. Average time per iteration: 1.9642964144275612
Iteration: 4229. Iteration Time: 2.228034019470215. Average time per iteration: 1.9647475556430654
Iteration: 4230. Iteration Time: 1.4489161968231201. Average time per iteration: 1.9638675154272607
Iteration: 4231. Iteration Time: 1.837162971496582. Average time per iteration: 1.963651865466896
Iteration: 4232. Iteration Time: 3.0615570545196533. Average time per iteration: 1.9655193435091551
Iteration: 4233. Iteration Time: 1.8161027431488037. Average time per iteration: 1.9652661087152712
Iteration: 4234. Iteration Time: 1.1197357177734375. Average time per iteration: 1.9638332205303646
Iteration: 4235. Iteration Time: 2.916203022003174. Average time per iteration: 1.965444969445114
Iteration: 4236. Iteration Time: 1.5780911445617676. Average time per iteration: 1.9647908597379118
Itera

Iteration: 4310. Iteration Time: 2.2286007404327393. Average time per iteration: 1.9707468905606427
Iteration: 4311. Iteration Time: 3.1593728065490723. Average time per iteration: 1.9725291425141616
Iteration: 4312. Iteration Time: 1.2920348644256592. Average time per iteration: 1.971510735100615
Iteration: 4313. Iteration Time: 1.5297260284423828. Average time per iteration: 1.9708505350378063
Iteration: 4314. Iteration Time: 2.7204229831695557. Average time per iteration: 1.9719694685580125
Iteration: 4315. Iteration Time: 3.695359945297241. Average time per iteration: 1.9745380430747428
Iteration: 4316. Iteration Time: 1.8158369064331055. Average time per iteration: 1.97430208360865
Iteration: 4317. Iteration Time: 1.6178197860717773. Average time per iteration: 1.9737725810522841
Iteration: 4318. Iteration Time: 2.504696846008301. Average time per iteration: 1.9745604719535534
Iteration: 4319. Iteration Time: 1.8404669761657715. Average time per iteration: 1.9743620014190675
Itera

Iteration: 4393. Iteration Time: 1.8568329811096191. Average time per iteration: 1.9849447462364573
Iteration: 4394. Iteration Time: 2.056382894515991. Average time per iteration: 1.9850401493708292
Iteration: 4395. Iteration Time: 3.04374623298645. Average time per iteration: 1.9864500464199386
Iteration: 4396. Iteration Time: 2.0866851806640625. Average time per iteration: 1.9865835356585524
Iteration: 4397. Iteration Time: 1.8702871799468994. Average time per iteration: 1.9864292442402833
Iteration: 4398. Iteration Time: 1.92787766456604. Average time per iteration: 1.9863517489926568
Iteration: 4399. Iteration Time: 1.1917800903320312. Average time per iteration: 1.9852995149347166
Iteration: 4400. Iteration Time: 2.880485773086548. Average time per iteration: 1.9864837776416193
Iteration: 4401. Iteration Time: 1.701502799987793. Average time per iteration: 1.9861074608887201
Iteration: 4402. Iteration Time: 1.6702980995178223. Average time per iteration: 1.9856909985278086
Iterati

Iteration: 4476. Iteration Time: 3.121767044067383. Average time per iteration: 1.9975271540192456
Iteration: 4477. Iteration Time: 1.812790870666504. Average time per iteration: 1.9973056230510697
Iteration: 4478. Iteration Time: 1.5387299060821533. Average time per iteration: 1.9967559773287327
Iteration: 4479. Iteration Time: 1.5960838794708252. Average time per iteration: 1.9962762909735035
Iteration: 4480. Iteration Time: 1.9861817359924316. Average time per iteration: 1.9962643826977489
Iteration: 4481. Iteration Time: 1.271430253982544. Average time per iteration: 1.9953986091568194
Iteration: 4482. Iteration Time: 1.301527976989746. Average time per iteration: 1.9945707386604072
Iteration: 4483. Iteration Time: 2.3761589527130127. Average time per iteration: 1.9950257163223977
Iteration: 4484. Iteration Time: 1.3783979415893555. Average time per iteration: 1.9942917891911098
Iteration: 4485. Iteration Time: 1.2831439971923828. Average time per iteration: 1.9934463449947615
Iter

Iteration: 4559. Iteration Time: 1.6175940036773682. Average time per iteration: 1.9896392819659958
Iteration: 4560. Iteration Time: 1.0397210121154785. Average time per iteration: 1.9886023852502415
Iteration: 4561. Iteration Time: 1.504640817642212. Average time per iteration: 1.988074756890633
Iteration: 4562. Iteration Time: 2.5254249572753906. Average time per iteration: 1.9886602526916137
Iteration: 4563. Iteration Time: 1.6744060516357422. Average time per iteration: 1.9883184394068505
Iteration: 4564. Iteration Time: 1.5058350563049316. Average time per iteration: 1.9877941706906195
Iteration: 4565. Iteration Time: 2.3733901977539062. Average time per iteration: 1.9882129760829168
Iteration: 4566. Iteration Time: 1.7292380332946777. Average time per iteration: 1.987932223301389
Iteration: 4567. Iteration Time: 1.38932204246521. Average time per iteration: 1.98728381257145
Iteration: 4568. Iteration Time: 1.3515739440917969. Average time per iteration: 1.9865959511174784
Iterati

In [41]:
df_pergame.to_csv("per_game.csv")

In [42]:
df_advanced.to_csv('rookie_advanced.csv')

In [43]:
df_per100.to_csv('per_100.csv')

In [44]:
career_pergame.to_csv('career_pergame.csv')

In [45]:
career_per100.to_csv('career_per100.csv')

In [46]:
career_advanced.to_csv('career_advanced.csv')

In [47]:
career_awards.to_csv('career_awards.csv')

In [48]:
draft_pick.to_csv('draft_pick.csv')